In [1]:
%matplotlib inline

import pandas as pd


pd.options.display.max_columns = 100
pd.options.display.max_rows = 300

In [2]:
ichiro_atbat_2015 = pd.read_csv('./output/datasets/ichiro_suzuki_2015_atbats.csv')
ichiro_pitchfx_2015 = pd.read_csv('./output/datasets/ichiro_suzuki_2015_pitchfx.csv')
ichiro_atbat_2016 = pd.read_csv('./output/datasets/ichiro_suzuki_2016_atbats.csv')
ichiro_pitchfx_2016 = pd.read_csv('./output/datasets/ichiro_suzuki_2016_pitchfx.csv')

In [3]:
ichiro_atbat_2015

,Unnamed: 0,Unnamed: 0.1,retro_game_id,year,month,day,st_fl,regseason_fl,playoff_fl,game_type,game_type_des,local_game_time,game_id,home_team_id,away_team_id,home_team_lg,away_team_lg,interleague_fl,park_id,park_name,park_location,inning_number,bat_home_id,outs_ct,pit_mlbid,pit_first_name,pit_last_name,pit_box_name,pit_hand_cd,bat_mlbid,bat_first_name,bat_last_name,bat_box_name,bat_hand_cd,ab_number,start_bases,end_bases,event_outs_ct,ab_des,event_tx,event_cd,hit_x,hit_y,ball_ct,strike_ct,pitch_seq,pitch_type_seq,battedball_cd
0,4232,123,MIA201504060,2015,4,6,F,T,F,R,Regular Season,16:10,413650,mia,atl,NL,NL,F,4169,Marlins Park,"Miami, FL",8,1,0,462382,Jim,Johnson,"Johnson, J",R,400085,Ichiro,Suzuki,"Suzuki, I",L,59,___,___,1,Ichiro Suzuki grounds out softly to first base...,Groundout,2,144.92,174.97,2,0,BBX,FT|FT|FF,G
1,5221,123,MIA201504070,2015,4,7,F,T,F,R,Regular Season,19:10,413665,mia,atl,NL,NL,F,4169,Marlins Park,"Miami, FL",7,1,2,592533,Cody,Martin,"Martin, Co",R,400085,Ichiro,Suzuki,"Suzuki, I",L,61,___,___,3,"Ichiro Suzuki grounds out, shortstop Andrelton...",Groundout,2,114.86,160.22,0,1,SX,FF|FT,G
2,5743,128,MIA201504080,2015,4,8,F,T,F,R,Regular Season,19:10,413672,mia,atl,NL,NL,F,4169,Marlins Park,"Miami, FL",7,1,2,501593,Luis,Avilan,Avilan,L,400085,Ichiro,Suzuki,"Suzuki, I",L,53,___,1__,2,Ichiro Suzuki singles on a bunt ground ball to...,Single,20,120.70,192.78,1,0,BX,FT|FT,G
3,8460,1057,MIA201504100,2015,4,10,F,T,F,R,Regular Season,19:10,413708,mia,tba,NL,AL,T,4169,Marlins Park,"Miami, FL",8,1,2,457117,Ernesto,Frieri,Frieri,R,400085,Ichiro,Suzuki,"Suzuki, I",L,76,12_,___,3,"Ichiro Suzuki grounds out, second baseman Loga...",Groundout,2,142.77,160.73,0,1,SX,FF|FF,G
4,9576,955,MIA201504110,2015,4,11,F,T,F,R,Regular Season,16:10,413723,mia,tba,NL,AL,T,4169,Marlins Park,"Miami, FL",6,1,1,502042,Chris,Archer,Archer,R,400085,Ichiro,Suzuki,"Suzuki, I",L,41,___,1__,1,Ichiro Suzuki walks.,Walk,14,NaN,NaN,4,0,BBBB,FF|FF|FF|FF,NaN
5,10772,1017,MIA201504120,2015,4,12,F,T,F,R,Regular Season,13:10,413738,mia,tba,NL,AL,T,4169,Marlins Park,"Miami, FL",2,1,0,501992,Nathan,Karns,Karns,R,400085,Ichiro,Suzuki,"Suzuki, I",L,12,___,___,1,Ichiro Suzuki flies out softly to left fielder...,Flyout,2,84.81,115.97,1,0,BX,U|U,F
6,10790,1035,MIA201504120,2015,4,12,F,T,F,R,Regular Season,13:10,413738,mia,tba,NL,AL,T,4169,Marlins Park,"Miami, FL",4,1,1,501992,Nathan,Karns,Karns,R,400085,Ichiro,Suzuki,"Suzuki, I",L,30,__3,___,2,Ichiro Suzuki out on a sacrifice fly to left f...,Sac Fly,2,NaN,NaN,2,2,SBSSBX,FF|FF|FF|FF|FF|FF,F
7,10812,1057,MIA201504120,2015,4,12,F,T,F,R,Regular Season,13:10,413738,mia,tba,NL,AL,T,4169,Marlins Park,"Miami, FL",6,1,2,501992,Nathan,Karns,Karns,R,400085,Ichiro,Suzuki,"Suzuki, I",L,52,___,___,3,Ichiro Suzuki strikes out swinging.,Strikeout,3,NaN,NaN,1,3,BSSS,FF|FF|KC|KC,NaN
8,10829,1074,MIA201504120,2015,4,12,F,T,F,R,Regular Season,13:10,413738,mia,tba,NL,AL,T,4169,Marlins Park,"Miami, FL",8,1,2,489446,Kirby,Yates,Yates,R,400085,Ichiro,Suzuki,"Suzuki, I",L,69,_2_,1_3,2,Ichiro Suzuki singles on a ground ball to left...,Single,20,75.86,123.60,0,2,SSX,FF|FF|FF,G
9,11533,522,ATL201504130,2015,4,13,F,T,F,R,Regular Season,19:10,413747,atl,mia,NL,NL,F,16,Turner Field,"Atlanta, GA",8,0,0,462382,Jim,Johnson,"Johnson, J",R,400085,Ichiro,Suzuki,"Suzuki, I",L,59,___,___,1,Ichiro Suzuki grounds out to first baseman Fre...,Groundout,2,148.45,172.94,2,1,BBSX,FF|CH|FF|FF,G


In [4]:
ichiro_atbat_2016

,Unnamed: 0,Unnamed: 0.1,retro_game_id,year,month,day,st_fl,regseason_fl,playoff_fl,game_type,game_type_des,local_game_time,game_id,home_team_id,away_team_id,home_team_lg,away_team_lg,interleague_fl,park_id,park_name,park_location,inning_number,bat_home_id,outs_ct,pit_mlbid,pit_first_name,pit_last_name,pit_box_name,pit_hand_cd,bat_mlbid,bat_first_name,bat_last_name,bat_box_name,bat_hand_cd,ab_number,start_bases,end_bases,event_outs_ct,ab_des,event_tx,event_cd,hit_x,hit_y,ball_ct,strike_ct,pitch_seq,pitch_type_seq,battedball_cd
0,3315,367,MIA201604050,2016,4,5,F,T,F,R,Regular Season,19:10,446883,mia,det,NL,AL,T,4169,Marlins Park,"Miami, FL",6,1,0,434378,Justin,Verlander,Verlander,R,400085,Ichiro,Suzuki,"Suzuki, I",L,45,___,___,1,Ichiro Suzuki pops out to pitcher Justin Verla...,Pop Out,2,147.44,193.28,3,2,SSSBBSSBX,FF|FF|FF|CU|CU|FF|FF|FF|SL,P
1,9032,538,NYN201604110,2016,4,11,F,T,F,R,Regular Season,19:10,446963,nyn,mia,NL,NL,F,3289,Citi Field,"Flushing, NY",9,0,1,460283,Jerry,Blevins,Blevins,L,400085,Ichiro,Suzuki,"Suzuki, I",L,80,___,___,2,"Ichiro Suzuki grounds out, shortstop Wilmer Fl...",Groundout,2,114.72,167.34,0,2,SSSX,SI|SI|SI|CU,G
2,9826,486,NYN201604120,2016,4,12,F,T,F,R,Regular Season,19:10,446974,nyn,mia,NL,NL,F,3289,Citi Field,"Flushing, NY",7,0,2,592789,Noah,Syndergaard,Syndergaard,R,400085,Ichiro,Suzuki,"Suzuki, I",L,51,_2_,___,3,Ichiro Suzuki singles on a ground ball to shor...,Single,20,107.62,126.65,0,1,SX,SI|SI,G
3,10865,650,NYN201604130,2016,4,13,F,T,F,R,Regular Season,13:10,446990,nyn,mia,NL,NL,F,3289,Citi Field,"Flushing, NY",1,0,1,548337,Logan,Verrett,Verrett,R,400085,Ichiro,Suzuki,"Suzuki, I",L,2,___,___,2,Ichiro Suzuki flies out to left fielder Yoenis...,Flyout,2,77.67,127.67,1,1,BSX,FF|CU|CU,F
4,10883,668,NYN201604130,2016,4,13,F,T,F,R,Regular Season,13:10,446990,nyn,mia,NL,NL,F,3289,Citi Field,"Flushing, NY",4,0,0,548337,Logan,Verrett,Verrett,R,400085,Ichiro,Suzuki,"Suzuki, I",L,20,___,_2_,0,Ichiro Suzuki doubles (1) on a line drive to c...,Double,21,152.29,48.32,0,2,SSX,FT|SL|CU,L
5,10902,687,NYN201604130,2016,4,13,F,T,F,R,Regular Season,13:10,446990,nyn,mia,NL,NL,F,3289,Citi Field,"Flushing, NY",6,0,0,548337,Logan,Verrett,Verrett,R,400085,Ichiro,Suzuki,"Suzuki, I",L,39,___,___,1,"Ichiro Suzuki grounds out, first baseman Wilme...",Groundout,2,143.15,170.40,3,2,SBSBBX,CH|FT|CH|CU|SL|FF,G
6,10919,704,NYN201604130,2016,4,13,F,T,F,R,Regular Season,13:10,446990,nyn,mia,NL,NL,F,3289,Citi Field,"Flushing, NY",8,0,0,460283,Jerry,Blevins,Blevins,L,400085,Ichiro,Suzuki,"Suzuki, I",L,56,___,___,1,Ichiro Suzuki flies out to left fielder Yoenis...,Flyout,2,67.51,126.14,2,2,SSSBSBSSSSX,CH|CH|CU|CU|CU|SI|SL|SI|SL|SL|SI,F
7,14694,220,MIA201604170,2016,4,17,F,T,F,R,Regular Season,13:10,447038,mia,atl,NL,NL,F,4169,Marlins Park,"Miami, FL",6,1,0,468504,Jhoulys,Chacin,Chacin,R,400085,Ichiro,Suzuki,"Suzuki, I",L,46,1__,1_3,0,Ichiro Suzuki singles on a ground ball to cent...,Single,20,127.92,104.27,1,2,BSSX,FT|FT|CU|FT,G
8,14708,234,MIA201604170,2016,4,17,F,T,F,R,Regular Season,13:10,447038,mia,atl,NL,NL,F,4169,Marlins Park,"Miami, FL",7,1,1,447714,Eric,O'Flaherty,O'Flaherty,L,400085,Ichiro,Suzuki,"Suzuki, I",L,60,_2_,1_3,1,Ichiro Suzuki walks.,Walk,14,NaN,NaN,4,2,BSBSBB,SI|SI|SI|SI|SI|SI,NaN
9,14723,249,MIA201604170,2016,4,17,F,T,F,R,Regular Season,13:10,447038,mia,atl,NL,NL,F,4169,Marlins Park,"Miami, FL",9,1,2,276351,Jason,Grilli,Grilli,R,400085,Ichiro,Suzuki,"Suzuki, I",L,75,_2_,_2_,2,Ichiro Suzuki singles on a line drive to left ...,Single,20,73.10,111.90,0,2,SSX,FF|FF|FF,L


In [5]:
ichiro_atbat_2015[ichiro_atbat_2015['event_cd'].isin([20, 21, 22, 23])]

,Unnamed: 0,Unnamed: 0.1,retro_game_id,year,month,day,st_fl,regseason_fl,playoff_fl,game_type,game_type_des,local_game_time,game_id,home_team_id,away_team_id,home_team_lg,away_team_lg,interleague_fl,park_id,park_name,park_location,inning_number,bat_home_id,outs_ct,pit_mlbid,pit_first_name,pit_last_name,pit_box_name,pit_hand_cd,bat_mlbid,bat_first_name,bat_last_name,bat_box_name,bat_hand_cd,ab_number,start_bases,end_bases,event_outs_ct,ab_des,event_tx,event_cd,hit_x,hit_y,ball_ct,strike_ct,pitch_seq,pitch_type_seq,battedball_cd
2,5743,128,MIA201504080,2015,4,8,F,T,F,R,Regular Season,19:10,413672,mia,atl,NL,NL,F,4169,Marlins Park,"Miami, FL",7,1,2,501593,Luis,Avilan,Avilan,L,400085,Ichiro,Suzuki,"Suzuki, I",L,53,___,1__,2,Ichiro Suzuki singles on a bunt ground ball to...,Single,20,120.70,192.78,1,0,BX,FT|FT,G
8,10829,1074,MIA201504120,2015,4,12,F,T,F,R,Regular Season,13:10,413738,mia,tba,NL,AL,T,4169,Marlins Park,"Miami, FL",8,1,2,489446,Kirby,Yates,Yates,R,400085,Ichiro,Suzuki,"Suzuki, I",L,69,_2_,1_3,2,Ichiro Suzuki singles on a ground ball to left...,Single,20,75.86,123.60,0,2,SSX,FF|FF|FF,G
14,12560,495,ATL201504140,2015,4,14,F,T,F,R,Regular Season,19:10,413761,atl,mia,NL,NL,F,16,Turner Field,"Atlanta, GA",8,0,1,605374,Andrew,McKirahan,McKirahan,L,400085,Ichiro,Suzuki,"Suzuki, I",L,68,1__,12_,1,Ichiro Suzuki singles on a ground ball to thir...,Single,20,101.86,177.01,1,2,BSSSX,SL|FF|FF|FF|SL,G
19,14375,236,NYN201504160,2015,4,16,F,T,F,R,Regular Season,19:10,413785,nyn,mia,NL,NL,F,3289,Citi Field,"Flushing, NY",7,0,1,606160,Rafael,Montero,"Montero, R",R,400085,Ichiro,Suzuki,"Suzuki, I",L,52,___,__3,1,Ichiro Suzuki triples (1) on a fly ball to rig...,Triple,22,164.38,57.43,3,2,BSBSBSSSX,CH|CH|SL|CH|FT|FT|FT|FF|CH,F
22,15261,653,NYN201504170,2015,4,17,F,T,F,R,Regular Season,19:10,413797,nyn,mia,NL,NL,F,3289,Citi Field,"Flushing, NY",7,0,1,112526,Bartolo,Colon,Colon,R,400085,Ichiro,Suzuki,"Suzuki, I",L,49,1__,12_,1,Ichiro Suzuki singles on a ground ball to left...,Single,20,90.71,98.59,0,1,SX,CH|FT,G
27,19269,664,PHI201504210,2015,4,21,F,T,F,R,Regular Season,19:05,413851,phi,mia,NL,NL,F,2681,Citizens Bank Park,"Philadelphia, PA",5,0,0,425532,Jerome,Williams,"Williams, J",R,400085,Ichiro,Suzuki,"Suzuki, I",L,33,___,1__,0,Ichiro Suzuki singles on a sharp ground ball t...,Single,20,90.76,126.05,2,0,BBX,FC|CH|SI,G
28,19281,676,PHI201504210,2015,4,21,F,T,F,R,Regular Season,19:05,413851,phi,mia,NL,NL,F,2681,Citizens Bank Park,"Philadelphia, PA",6,0,1,425532,Jerome,Williams,"Williams, J",R,400085,Ichiro,Suzuki,"Suzuki, I",L,45,1__,12_,1,Ichiro Suzuki singles on a soft line drive to ...,Single,20,91.31,120.60,1,1,SBX,FC|FF|SI,NaN
33,20420,634,PHI201504220,2015,4,22,F,T,F,R,Regular Season,19:05,413866,phi,mia,NL,NL,F,2681,Citizens Bank Park,"Philadelphia, PA",8,0,2,571704,Ken,Giles,Giles,R,400085,Ichiro,Suzuki,"Suzuki, I",L,58,___,1__,2,Ichiro Suzuki singles on a soft ground ball to...,Single,20,130.05,165.44,1,2,SSSBX,FF|SL|SL|FF|FF,G
38,21501,594,PHI201504230,2015,4,23,F,T,F,R,Regular Season,13:05,413880,phi,mia,NL,NL,F,2681,Citizens Bank Park,"Philadelphia, PA",7,0,0,518603,Justin,DeFratus,DeFratus,R,400085,Ichiro,Suzuki,"Suzuki, I",L,56,___,1__,0,Ichiro Suzuki singles on a ground ball to left...,Single,20,107.46,142.41,2,1,SBBX,FT|CH|FT|FT,G
41,22884,1080,MIA201504240,2015,4,24,F,T,F,R,Regular Season,19:10,413899,mia,was,NL,NL,F,4169,Marlins Park,"Miami, FL",5,1,2,519455,Jordan,Zimmermann,Zimmermann,R,400085,Ichiro,Suzuki,"Suzuki, I",L,34,___,1__,2,Ichiro Suzuki singles on a sharp ground ball t...,Single,20,119.26,156.27,1,1,BSX,FF|FF|FF,G


In [7]:
ichiro_atbat_2016[ichiro_atbat_2016['event_cd'].isin([20, 21, 22, 23])]

,Unnamed: 0,Unnamed: 0.1,retro_game_id,year,month,day,st_fl,regseason_fl,playoff_fl,game_type,game_type_des,local_game_time,game_id,home_team_id,away_team_id,home_team_lg,away_team_lg,interleague_fl,park_id,park_name,park_location,inning_number,bat_home_id,outs_ct,pit_mlbid,pit_first_name,pit_last_name,pit_box_name,pit_hand_cd,bat_mlbid,bat_first_name,bat_last_name,bat_box_name,bat_hand_cd,ab_number,start_bases,end_bases,event_outs_ct,ab_des,event_tx,event_cd,hit_x,hit_y,ball_ct,strike_ct,pitch_seq,pitch_type_seq,battedball_cd
2,9826,486,NYN201604120,2016,4,12,F,T,F,R,Regular Season,19:10,446974,nyn,mia,NL,NL,F,3289,Citi Field,"Flushing, NY",7,0,2,592789,Noah,Syndergaard,Syndergaard,R,400085,Ichiro,Suzuki,"Suzuki, I",L,51,_2_,___,3,Ichiro Suzuki singles on a ground ball to shor...,Single,20,107.62,126.65,0,1,SX,SI|SI,G
4,10883,668,NYN201604130,2016,4,13,F,T,F,R,Regular Season,13:10,446990,nyn,mia,NL,NL,F,3289,Citi Field,"Flushing, NY",4,0,0,548337,Logan,Verrett,Verrett,R,400085,Ichiro,Suzuki,"Suzuki, I",L,20,___,_2_,0,Ichiro Suzuki doubles (1) on a line drive to c...,Double,21,152.29,48.32,0,2,SSX,FT|SL|CU,L
7,14694,220,MIA201604170,2016,4,17,F,T,F,R,Regular Season,13:10,447038,mia,atl,NL,NL,F,4169,Marlins Park,"Miami, FL",6,1,0,468504,Jhoulys,Chacin,Chacin,R,400085,Ichiro,Suzuki,"Suzuki, I",L,46,1__,1_3,0,Ichiro Suzuki singles on a ground ball to cent...,Single,20,127.92,104.27,1,2,BSSX,FT|FT|CU|FT,G
9,14723,249,MIA201604170,2016,4,17,F,T,F,R,Regular Season,13:10,447038,mia,atl,NL,NL,F,4169,Marlins Park,"Miami, FL",9,1,2,276351,Jason,Grilli,Grilli,R,400085,Ichiro,Suzuki,"Suzuki, I",L,75,_2_,_2_,2,Ichiro Suzuki singles on a line drive to left ...,Single,20,73.10,111.90,0,2,SSX,FF|FF|FF,L
11,17235,1139,MIA201604190,2016,4,19,F,T,F,R,Regular Season,19:10,447073,mia,was,NL,NL,F,4169,Marlins Park,"Miami, FL",9,1,1,518875,Shawn,Kelley,Kelley,R,400085,Ichiro,Suzuki,"Suzuki, I",L,70,1__,12_,1,Ichiro Suzuki singles on a ground ball to thir...,Single,20,102.99,166.33,0,0,X,FF,G
16,19318,954,MIA201604210,2016,4,21,F,T,F,R,Regular Season,13:10,447102,mia,was,NL,NL,F,4169,Marlins Park,"Miami, FL",1,1,0,453286,Max,Scherzer,Scherzer,R,400085,Ichiro,Suzuki,"Suzuki, I",L,5,___,1__,0,Ichiro Suzuki singles on a ground ball to cent...,Single,20,145.99,100.20,1,0,BX,FF|FF,G
18,19352,988,MIA201604210,2016,4,21,F,T,F,R,Regular Season,13:10,447102,mia,was,NL,NL,F,4169,Marlins Park,"Miami, FL",5,1,0,453286,Max,Scherzer,Scherzer,R,400085,Ichiro,Suzuki,"Suzuki, I",L,39,___,1__,0,Ichiro Suzuki singles on a line drive to left ...,Single,20,87.06,115.46,2,2,BSBSSSSSX,FF|FF|CH|FF|CU|SL|FF|FF|CH,L
21,24498,591,LAN201604260,2016,4,26,F,T,F,R,Regular Season,19:10,447168,lan,mia,NL,NL,F,22,Dodger Stadium,"Los Angeles, CA",8,0,2,488786,Louis,Coleman,Coleman,R,400085,Ichiro,Suzuki,"Suzuki, I",L,64,1__,12_,2,Ichiro Suzuki singles on a line drive to left ...,Single,20,92.90,106.31,1,2,SSBSSX,FF|SL|FF|SL|FF|SL,L
27,27292,649,MIL201604290,2016,4,29,F,T,F,R,Regular Season,19:10,447208,mil,mia,NL,NL,F,32,Miller Park,"Milwaukee, WI",1,0,0,605200,Zach,Davies,Davies,R,400085,Ichiro,Suzuki,"Suzuki, I",L,1,___,1__,0,Ichiro Suzuki singles on a ground ball to righ...,Single,20,188.03,108.75,1,0,BX,SI|SI,G
31,27357,714,MIL201604290,2016,4,29,F,T,F,R,Regular Season,19:10,447208,mil,mia,NL,NL,F,32,Miller Park,"Milwaukee, WI",9,0,2,592341,David,Goforth,Goforth,R,400085,Ichiro,Suzuki,"Suzuki, I",L,66,_2_,_2_,2,Ichiro Suzuki singles on a ground ball to left...,Single,20,79.79,105.20,2,0,BBX,FF|FT|FT,G


In [8]:
ichiro_pitchfx_2015[ichiro_pitchfx_2015['pa_event_cd'].isin([20, 21, 22, 23])]

,Unnamed: 0,Unnamed: 0.1,retro_game_id,year,month,day,st_fl,regseason_fl,playoff_fl,game_type,game_type_des,local_game_time,game_id,home_team_id,away_team_id,home_team_lg,away_team_lg,interleague_fl,park_id,park_name,park_location,inning_number,bat_home_id,outs_ct,pit_mlbid,pit_first_name,pit_last_name,pit_box_name,pit_hand_cd,bat_mlbid,bat_first_name,bat_last_name,bat_box_name,bat_hand_cd,ab_number,start_bases,end_bases,event_outs_ct,pa_ball_ct,pa_strike_ct,pitch_seq,pa_terminal_fl,pa_event_cd,pitch_res,pitch_des,pitch_id,x,y,start_speed,end_speed,sz_top,sz_bot,pfx_x,pfx_z,px,pz,x0,y0,z0,vx0,vy0,vz0,ax,ay,az,break_y,break_angle,break_length,pitch_type,pitch_type_seq,type_confidence,zone,spin_dir,spin_rate,sv_id
5,17762,486,MIA201504080,2015,4,8,F,T,F,R,Regular Season,19:10,413672,mia,atl,NL,NL,F,4169,Marlins Park,"Miami, FL",7,1,2,501593,Luis,Avilan,Avilan,L,400085,Ichiro,Suzuki,"Suzuki, I",L,53,___,1__,2,0,0,B,F,20,B,Ball,429,72.67,173.36,93.4,85.5,3.43,1.50,12.49,6.13,1.160,2.420,1.400,50.0,5.620,-5.000,-136.800,-4.780,23.470,32.580,-20.580,23.8,-45.8,6.7,FT,FT,2.000,14.0,116.290,2776.159,150408_211809
6,17763,487,MIA201504080,2015,4,8,F,T,F,R,Regular Season,19:10,413672,mia,atl,NL,NL,F,4169,Marlins Park,"Miami, FL",7,1,2,501593,Luis,Avilan,Avilan,L,400085,Ichiro,Suzuki,"Suzuki, I",L,53,___,1__,2,1,0,BX,T,20,X,"In play, no out",430,79.61,160.78,92.9,85.1,3.43,1.50,13.20,4.64,0.980,2.890,1.390,50.0,5.590,-5.670,-136.050,-2.840,24.570,31.690,-23.460,23.8,-44.0,7.3,FT,FT|FT,2.000,12.0,109.528,2779.946,150408_211824
25,37072,4015,MIA201504120,2015,4,12,F,T,F,R,Regular Season,13:10,413738,mia,tba,NL,AL,T,4169,Marlins Park,"Miami, FL",8,1,2,489446,Kirby,Yates,Yates,R,400085,Ichiro,Suzuki,"Suzuki, I",L,69,_2_,1_3,2,0,0,S,F,20,S,Called Strike,591,144.29,155.05,90.7,82.6,3.50,1.50,-8.85,7.30,-0.716,3.101,-0.886,50.0,5.370,3.439,-132.909,-2.250,-15.647,31.589,-19.192,23.7,37.0,5.7,FF,FF,2.000,11.0,230.317,2219.282,150412_155655
26,37073,4016,MIA201504120,2015,4,12,F,T,F,R,Regular Season,13:10,413738,mia,tba,NL,AL,T,4169,Marlins Park,"Miami, FL",8,1,2,489446,Kirby,Yates,Yates,R,400085,Ichiro,Suzuki,"Suzuki, I",L,69,_2_,1_3,2,0,1,SS,F,20,S,Foul,592,124.17,165.64,91.6,84.3,3.73,1.50,-6.48,9.34,-0.188,2.709,-0.934,50.0,5.309,4.206,-134.229,-4.045,-11.818,29.133,-15.071,23.8,34.4,4.3,FF,FF|FF,2.000,5.0,214.643,2246.917,150412_155716
27,37074,4017,MIA201504120,2015,4,12,F,T,F,R,Regular Season,13:10,413738,mia,tba,NL,AL,T,4169,Marlins Park,"Miami, FL",8,1,2,489446,Kirby,Yates,Yates,R,400085,Ichiro,Suzuki,"Suzuki, I",L,69,_2_,1_3,2,0,2,SSX,T,20,X,"In play, no out",593,164.49,164.45,92.0,83.0,3.73,1.50,-6.45,7.48,-1.246,2.753,-1.077,50.0,5.319,1.755,-134.945,-3.229,-11.600,36.072,-18.651,23.7,29.7,5.1,FF,FF|FF|FF,2.000,11.0,220.622,1915.448,150412_155746
46,43653,1877,ATL201504140,2015,4,14,F,T,F,R,Regular Season,19:10,413761,atl,mia,NL,NL,F,16,Turner Field,"Atlanta, GA",8,0,1,605374,Andrew,McKirahan,McKirahan,L,400085,Ichiro,Suzuki,"Suzuki, I",L,68,1__,12_,1,0,0,B,F,20,B,Ball,548,136.48,203.36,83.6,77.4,3.45,1.48,1.41,-2.93,-0.511,1.312,1.593,50.0,5.537,-5.519,-122.467,-2.630,2.133,25.170,-36.524,23.8,-1.8,9.9,SL,SL,2.000,13.0,26.119,573.931,150414_214136
47,43654,1878,ATL201504140,2015,4,14,F,T,F,R,Regular Season,19:10,413761,atl,mia,NL,NL,F,16,Turner Field,"Atlanta, GA",8,0,1,605374,Andrew,McKirahan,McKirahan,L,400085,Ichiro,Suzuki,"Suzuki, I",L,68,1__,12_,1,1,0,BS,F,20,S,Foul,549,104.80,165.58,93.4,86.3,3.56,1.48,0.22,9.04,0.320,2.711,1.428,50.0,5.555,-3.076,-136.932,-4.952,0.417,29.128,-14.877,23.8,0.8,3.4,FF,SL|FF,2.000,6.0,178.621,1833.134,150414_214152
48,43655,1879,ATL201504140,2015,4,14,F,T,F,R,Regular Season,19:10,413761,atl,mia,NL,NL,F,16,Turner Field,"Atlanta, GA",8,0,1,605374,Andrew,McKirahan,McKirahan,L,400085,Ichiro,Suzuki,"Suzuki, I",L,68,1__,12_,1,1,1,BSS,F,20,S,Called Strike,550,134.15,159.35,93.1,86.3,3.45,1.48,1.83,9.59,-0.450,2.942,1.085,50.0,5.672,-4.790,-136.366,-4.803,3.477,27.612,-13.889,23.8,-9.7,3.2,FF,SL|FF|FF,2.0

In [9]:
ichiro_pitchfx_2016[ichiro_pitchfx_2016['pa_event_cd'].isin([20, 21, 22, 23])]

,Unnamed: 0,Unnamed: 0.1,retro_game_id,year,month,day,st_fl,regseason_fl,playoff_fl,game_type,game_type_des,local_game_time,game_id,home_team_id,away_team_id,home_team_lg,away_team_lg,interleague_fl,park_id,park_name,park_location,inning_number,bat_home_id,outs_ct,pit_mlbid,pit_first_name,pit_last_name,pit_box_name,pit_hand_cd,bat_mlbid,bat_first_name,bat_last_name,bat_box_name,bat_hand_cd,ab_number,start_bases,end_bases,event_outs_ct,pa_ball_ct,pa_strike_ct,pitch_seq,pa_terminal_fl,pa_event_cd,pitch_res,pitch_des,pitch_id,x,y,start_speed,end_speed,sz_top,sz_bot,pfx_x,pfx_z,px,pz,x0,y0,z0,vx0,vy0,vz0,ax,ay,az,break_y,break_angle,break_length,pitch_type,pitch_type_seq,type_confidence,zone,spin_dir,spin_rate,sv_id
13,37050,1878,NYN201604120,2016,4,12,F,T,F,R,Regular Season,19:10,446974,nyn,mia,NL,NL,F,3289,Citi Field,"Flushing, NY",7,0,2,592789,Noah,Syndergaard,Syndergaard,R,400085,Ichiro,Suzuki,"Suzuki, I",L,51,_2_,___,3,0,0,S,F,20,S,Called Strike,412,157.63,192.18,97.1,88.5,3.35,1.42,-7.83,8.06,-1.066,1.726,-0.572,50.0,6.240,1.452,-142.140,-9.742,-15.771,37.177,-15.865,23.7,40.0,4.8,SI,SI,0.891,13.0,224.038,2315.615,160412_211318
14,37051,1879,NYN201604120,2016,4,12,F,T,F,R,Regular Season,19:10,446974,nyn,mia,NL,NL,F,3289,Citi Field,"Flushing, NY",7,0,2,592789,Noah,Syndergaard,Syndergaard,R,400085,Ichiro,Suzuki,"Suzuki, I",L,51,_2_,___,3,0,1,SX,T,20,X,"In play, out(s)",413,175.40,176.41,96.6,88.3,3.39,1.42,-6.21,8.89,-1.532,2.310,-0.611,50.0,6.278,-0.319,-141.466,-8.459,-12.455,35.584,-14.291,23.7,37.4,4.2,SI,SI|SI,0.882,13.0,214.855,2234.932,160412_211354
18,41207,2587,NYN201604130,2016,4,13,F,T,F,R,Regular Season,13:10,446990,nyn,mia,NL,NL,F,3289,Citi Field,"Flushing, NY",4,0,0,548337,Logan,Verrett,Verrett,R,400085,Ichiro,Suzuki,"Suzuki, I",L,20,___,_2_,0,0,0,S,F,21,S,Called Strike,141,157.75,167.91,87.9,79.6,3.51,1.42,-8.52,12.29,-1.069,2.625,-1.306,50.0,6.084,3.382,-128.750,-6.370,-14.034,31.496,-11.859,23.7,44.6,4.7,FT,FT,2.000,11.0,214.636,2782.250,160413_135324
19,41208,2588,NYN201604130,2016,4,13,F,T,F,R,Regular Season,13:10,446990,nyn,mia,NL,NL,F,3289,Citi Field,"Flushing, NY",4,0,0,548337,Logan,Verrett,Verrett,R,400085,Ichiro,Suzuki,"Suzuki, I",L,20,___,_2_,0,0,1,SS,F,21,S,Foul,142,132.97,192.64,81.5,74.7,3.37,1.42,-0.45,1.80,-0.419,1.709,-1.711,50.0,5.786,3.169,-119.495,-3.260,-0.647,26.169,-29.535,23.8,0.0,8.6,SL,FT|SL,2.000,7.0,193.774,329.888,160413_135337
20,41209,2589,NYN201604130,2016,4,13,F,T,F,R,Regular Season,13:10,446990,nyn,mia,NL,NL,F,3289,Citi Field,"Flushing, NY",4,0,0,548337,Logan,Verrett,Verrett,R,400085,Ichiro,Suzuki,"Suzuki, I",L,20,___,_2_,0,0,2,SSX,T,21,X,"In play, no out",143,118.94,212.89,77.2,70.9,3.37,1.42,0.08,-3.61,-0.051,0.959,-1.329,50.0,6.158,2.808,-113.144,-3.236,0.098,24.464,-36.687,23.8,-0.9,11.9,CU,FT|SL|CU,2.000,13.0,1.242,578.813,160413_135404
38,55783,793,MIA201604170,2016,4,17,F,T,F,R,Regular Season,13:10,447038,mia,atl,NL,NL,F,4169,Marlins Park,"Miami, FL",6,1,0,468504,Jhoulys,Chacin,Chacin,R,400085,Ichiro,Suzuki,"Suzuki, I",L,46,1__,1_3,0,0,0,B,F,20,B,Ball,368,178.37,173.90,86.8,78.7,3.53,1.42,-8.44,4.88,-1.610,2.403,-0.766,50.0,6.261,0.618,-127.174,-4.708,-13.503,31.662,-24.297,23.7,27.7,7.3,FT,FT,0.920,13.0,239.742,1783.344,160417_150311
39,55784,794,MIA201604170,2016,4,17,F,T,F,R,Regular Season,13:10,447038,mia,atl,NL,NL,F,4169,Marlins Park,"Miami, FL",6,1,0,468504,Jhoulys,Chacin,Chacin,R,400085,Ichiro,Suzuki,"Suzuki, I",L,46,1__,1_3,0,1,0,BS,F,20,S,Foul,369,133.85,184.13,87.6,80.1,3.37,1.42,-7.60,7.35,-0.442,2.024,-0.505,50.0,6.069,2.640,-128.332,-6.231,-12.509,29.628,-19.990,23.7,29.1,6.0,FT,FT|FT,0.916,7.0,225.753,1974.075,160417_150328
40,55785,795,MIA201604170,2016,4,17,F,T,F,R,Regular Season,13:10,447038,mia,atl,NL,NL,F,4169,Marlins Park,"Miami, FL",6,1,0,468504,Jhoulys,Chacin,Chacin,R,400085,Ichiro,Suzuki,"Suzuki, I",L,46,1__,1_3,0,1,1,BSS,F,20,S,Foul,370,120.13,182.78,77.2,71.2,3.37,1.42,7.38,-1.20,-0.082,2.074,-0.604,50.0,6.263,-0.968,-113.190,-1.762,9.474,22.642,-3

In [10]:
ichiro_atbat_2015.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 441 entries, 0 to 440
Data columns (total 48 columns):
Unnamed: 0         441 non-null int64
Unnamed: 0.1       441 non-null int64
retro_game_id      441 non-null object
year               441 non-null int64
month              441 non-null int64
day                441 non-null int64
st_fl              441 non-null object
regseason_fl       441 non-null object
playoff_fl         441 non-null object
game_type          441 non-null object
game_type_des      441 non-null object
local_game_time    441 non-null object
game_id            441 non-null int64
home_team_id       441 non-null object
away_team_id       441 non-null object
home_team_lg       441 non-null object
away_team_lg       441 non-null object
interleague_fl     441 non-null object
park_id            441 non-null int64
park_name          441 non-null object
park_location      441 non-null object
inning_number      441 non-null int64
bat_home_id        441 non-null int64
outs_ct

In [11]:
ichiro_pitchfx_2015.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1635 entries, 0 to 1634
Data columns (total 75 columns):
Unnamed: 0         1635 non-null int64
Unnamed: 0.1       1635 non-null int64
retro_game_id      1635 non-null object
year               1635 non-null int64
month              1635 non-null int64
day                1635 non-null int64
st_fl              1635 non-null object
regseason_fl       1635 non-null object
playoff_fl         1635 non-null object
game_type          1635 non-null object
game_type_des      1635 non-null object
local_game_time    1635 non-null object
game_id            1635 non-null int64
home_team_id       1635 non-null object
away_team_id       1635 non-null object
home_team_lg       1635 non-null object
away_team_lg       1635 non-null object
interleague_fl     1635 non-null object
park_id            1635 non-null int64
park_name          1635 non-null object
park_location      1635 non-null object
inning_number      1635 non-null int64
bat_home_id        16

In [13]:
ichiro_pitchfx_2015.shape


(1635, 75)

In [14]:
ichiro_pitchfx_2016.shape

(954, 75)

In [15]:
# イチローがヒットにした投球
pitchfx_hits_2015 = ichiro_pitchfx_2015.query("pitch_res == 'X' and pa_event_cd in [{}]".format(','.join(['20', '21', '22', '23', ])))
pitchfx_hits_2016 = ichiro_pitchfx_2016.query("pitch_res == 'X' and pa_event_cd in [{}]".format(','.join(['20', '21', '22', '23', ])))

In [16]:
# ヒットにした投球の平均球速と回転数
pitchfx_hits_2015[['start_speed', 'end_speed', 'spin_rate']].mean(numeric_only=True)

start_speed      88.816484
end_speed        81.559341
spin_rate      1813.112868
dtype: float64

In [17]:
# ヒットにした投球の平均球速と回転数
pitchfx_hits_2016[['start_speed', 'end_speed', 'spin_rate']].mean(numeric_only=True)

start_speed      88.406154
end_speed        81.506154
spin_rate      1826.277600
dtype: float64

In [18]:
# ヒットにした投球の球速と回転数を色々統計とってみる
pitchfx_hits_2015[['start_speed', 'end_speed', 'spin_rate']].describe()

,start_speed,end_speed,spin_rate
count,91.000000,91.000000,91.000000
mean,88.816484,81.559341,1813.112868
std,5.604844,5.017303,606.246627
min,75.500000,69.200000,174.172000
25%,85.600000,78.450000,1573.624000
50%,90.900000,83.200000,1977.150000
75%,92.900000,85.300000,2224.841000
max,97.000000,89.500000,2779.946000


In [19]:
# ヒットにした投球の球速と回転数を色々統計とってみる
pitchfx_hits_2016[['start_speed', 'end_speed', 'spin_rate']].describe()

,start_speed,end_speed,spin_rate
count,65.000000,65.000000,65.000000
mean,88.406154,81.506154,1826.277600
std,6.021988,5.385132,693.978934
min,74.500000,68.400000,170.475000
25%,83.500000,77.600000,1389.529000
50%,91.200000,83.600000,1974.543000
75%,92.900000,85.500000,2234.932000
max,96.600000,89.500000,3062.737000


In [20]:
# ヒットにした投球の平均球速と回転数(球種毎にGroup by)
pitchfx_hits_2015.groupby('pitch_type')[['start_speed', 'end_speed', 'spin_rate']].count()

,start_speed,end_speed,spin_rate
pitch_type,,,
CH,8,8,8
CU,6,6,6
FC,2,2,2
FF,40,40,40
FS,3,3,3
FT,14,14,14
SI,7,7,7
SL,11,11,11


In [21]:
# ヒットにした投球の平均球速と回転数(球種毎にGroup by)
pitchfx_hits_2016.groupby('pitch_type')[['start_speed', 'end_speed', 'spin_rate']].count()

,start_speed,end_speed,spin_rate
pitch_type,,,
CH,7,7,7
CU,5,5,5
FC,2,2,2
FF,27,27,27
FS,1,1,1
FT,8,8,8
KC,2,2,2
SI,5,5,5
SL,8,8,8


In [22]:
# ヒットにした投球の平均球速と回転数
pitchfx_hits_2015[['start_speed', 'end_speed', 'spin_rate']].mean(numeric_only=True)

start_speed      88.816484
end_speed        81.559341
spin_rate      1813.112868
dtype: float64

In [23]:
# ヒットにした投球の平均球速と回転数
pitchfx_hits_2016[['start_speed', 'end_speed', 'spin_rate']].mean(numeric_only=True)

start_speed      88.406154
end_speed        81.506154
spin_rate      1826.277600
dtype: float64

In [24]:
# ヒットにした投球の平均球速と回転数(ヒットの種類ごとにGroup by)
pitchfx_hits_2015.groupby(['pa_event_cd', 'pitch_type'])[['start_speed', 'end_speed', 'spin_rate']].describe()

start_speed  end_speed    spin_rate
pa_event_cd pitch_type                                           
20          CH         count     6.000000   6.000000     6.000000
                       mean     81.666667  75.166667  1682.839500
                       std       4.664190   4.410291   457.384627
                       min      75.500000  70.400000   856.126000
                       25%      78.800000  71.475000  1626.308250
                       50%      81.650000  74.950000  1740.783500
                       75%      84.050000  77.975000  1908.298000
                       max      88.500000  81.400000  2207.058000
            CU         count     5.000000   5.000000     5.000000
                       mean     78.800000  72.660000  1424.302000
                       std       1.838478   2.006988   501.140260
                       min      75.900000  69.200000   901.443000
                       25%      78.300000  72.700000  1087.278000
                       50%      79.300000  73.600000  1219.863000
                       75%      79.800000  73.600000  1887.857000
                       max      80.700000  74.200000  2025.069000
            FC         count     2.000000   2.000000     2.000000
                       mean     89.000000  82.100000  1001.973500
                       std       0.141421   0.141421   541.131142
                       min      88.900000  82.000000   619.336000
                       25%      88.950000  82.050000   810.654750
                       50%      89.000000  82.100000  1001.973500
                       75%      89.050000  82.150000  1193.292250
                       max      89.100000  82.200000  1384.611000
            FF         count    33.000000  33.000000    33.000000
                       mean     92.839394  85.078788  2149.604848
                       std       1.977204   1.911929   269.712484
                       min      89.200000  81.300000  1583.655000
                       25%      91.800000  84.000000  1971.020000
                       50%      92.700000  85.200000  2105.524000
                       75%      93.600000  86.000000  2351.114000
                       max      97.000000  89.500000  2660.508000
            FS         count     3.000000   3.000000     3.000000
                       mean     86.066667  79.366667  1710.180667
                       std       3.308071   3.324655   425.476061
                       min      82.900000  76.300000  1275.561000
                       25%      84.350000  77.600000  1502.330000
                       50%      85.800000  78.900000  1729.099000
                       75%      87.650000  80.900000  1927.490500
                       max      89.500000  82.900000  2125.882000
            FT         count    13.000000  13.000000    13.000000
                       mean     91.123077  83.700000  2061.634077
                       std       3.544986   3.516390   333.492643
                       min      85.800000  78.600000  1505.154000
                       25%      89.000000  81.400000  1964.709000
                       50%      91.100000  84.100000  2080.839000
                       75%      93.900000  86.600000  2119.686000
                       max      96.800000  89.100000  2779.946000
            SI         count     7.000000   7.000000     7.000000
                       mean     90.142857  82.414286  2077.175857
                       std       2.394339   1.770526   293.736399
                       min      87.600000  79.900000  1519.296000
                       25%      88.050000  81.300000  1985.287000
                       50%      89.600000  82.300000  2099.659000
                       75%      92.250000  83.450000  2264.283000
                       max      93.200000  85.200000  2422.136000
            SL         count    10.000000  10.000000    10.000000
                       mean     82.750000  76.380000   653.225700
                       std       3.362291   3.042221   419.289612
    

In [25]:
# ヒットにした投球の平均球速と回転数(ヒットの種類ごとにGroup by)
pitchfx_hits_2016.groupby(['pa_event_cd', 'pitch_type'])[['start_speed', 'end_speed', 'spin_rate']].describe()

start_speed  end_speed    spin_rate
pa_event_cd pitch_type                                           
20          CH         count     4.000000   4.000000     4.000000
                       mean     82.950000  76.800000  1858.811000
                       std       3.016068   2.708013    37.562366
                       min      78.700000  73.000000  1825.265000
                       25%      82.225000  76.150000  1832.876750
                       50%      83.650000  77.400000  1850.515500
                       75%      84.375000  78.050000  1876.449750
                       max      85.800000  79.400000  1908.948000
            CU         count     4.000000   4.000000     4.000000
                       mean     75.425000  69.500000  1488.271500
                       std       0.935860   0.969536   182.888036
                       min      74.500000  68.400000  1290.893000
                       25%      74.725000  68.850000  1373.197250
                       50%      75.350000  69.550000  1475.259500
                       75%      76.050000  70.200000  1590.333750
                       max      76.500000  70.500000  1711.674000
            FC         count     1.000000   1.000000     1.000000
                       mean     85.100000  79.700000   212.084000
                       std            NaN        NaN          NaN
                       min      85.100000  79.700000   212.084000
                       25%      85.100000  79.700000   212.084000
                       50%      85.100000  79.700000   212.084000
                       75%      85.100000  79.700000   212.084000
                       max      85.100000  79.700000   212.084000
            FF         count    24.000000  24.000000    24.000000
                       mean     92.591667  84.995833  2324.963875
                       std       1.763622   1.902282   380.282681
                       min      89.300000  81.800000  1235.117000
                       25%      91.300000  83.675000  2131.770000
                       50%      92.800000  85.200000  2287.500500
                       75%      94.050000  86.350000  2614.489750
                       max      95.200000  89.500000  3062.737000
            FS         count     1.000000   1.000000     1.000000
                       mean     83.200000  77.900000  1389.529000
                       std            NaN        NaN          NaN
                       min      83.200000  77.900000  1389.529000
                       25%      83.200000  77.900000  1389.529000
                       50%      83.200000  77.900000  1389.529000
                       75%      83.200000  77.900000  1389.529000
                       max      83.200000  77.900000  1389.529000
            FT         count     7.000000   7.000000     7.000000
                       mean     91.614286  84.128571  2127.629429
                       std       2.082009   2.088631   545.916934
                       min      87.100000  79.800000  1655.612000
                       25%      91.700000  84.000000  1698.235000
                       50%      92.300000  84.400000  1745.221000
                       75%      92.700000  85.150000  2637.721500
                       max      93.100000  86.400000  2820.660000
            KC         count     2.000000   2.000000     2.000000
                       mean     79.700000  74.050000   961.955000
                       std       1.131371   1.060660   270.789370
                       min      78.900000  73.300000   770.478000
                       25%      79.300000  73.675000   866.216500
                       50%      79.700000  74.050000   961.955000
                       75%      80.100000  74.425000  1057.693500
                       max      80.500000  74.800000  1153.432000
            SI         count     5.000000   5.000000     5.000000
                       mean     93.040000  85.700000  2132.890400
                       std       2.948389   2.699074    83.570790
    